In [ ]:
import sys, os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_ply

# Data structures and functions for rendering
from pytorch3d.structures import Meshes

In [ ]:
def verts_to_pts(verts):
    x = verts[:,0].numpy()
    y = verts[:,1].numpy()
    z = verts[:,2].numpy()
    return x,y,z

def get_min_max(data):
    data = data
    min_data = np.amin(data.flatten())
    max_data = np.amax(data.flatten())
    return min_data, max_data

def interpolate(data, degree):
    inter_data = []
    for k in range(data.shape[0]-1):
        for i in range(degree):
            inter_data.append(data[k, :] + (i/degree) * (data[k+1, :] - data[k, :]))
    inter_data.append(data[k+1, :])
    return np.stack(inter_data)

In [ ]:
# Set paths
DATA_DIR = "experiments/viz2"
CWD_PATH = os.path.dirname(os.path.abspath(os.getcwd()))
DATA_PATH = os.path.join(CWD_PATH, DATA_DIR)

files = []
files_number = np.array([])
files_order = np.array([])

for ply_file in os.listdir(DATA_PATH):
    files.append(ply_file)
    num = ''
    for c in ply_file:
        if c.isdigit():
            num = num + c
    num = num[5:-1]
    files_number = np.append(files_number, int(num))
files_order = np.argsort(files_number)
print(files_order)
print(type(files_order))

In [ ]:
x = []
y = []
z = []

for _, ordered_Index in enumerate(files_order): 
    ply_file = files[ordered_Index]   
    print("Load File ", ply_file)
    verts, faces = load_ply(os.path.join(DATA_PATH, ply_file))
    x_temp, y_temp, z_temp = verts_to_pts(verts)
    x.append(x_temp)
    y.append(y_temp)
    z.append(z_temp)

In [ ]:
x = np.stack(x)
y = np.stack(y)
z = np.stack(z)

inter_degree = 10
x = interpolate(x, inter_degree)
y = interpolate(y, inter_degree)
z = interpolate(z, inter_degree)

xm, xM = get_min_max(x)
ym, yM = get_min_max(y)
zm, zM = get_min_max(z)

In [ ]:
nth = 3
fig = go.Figure(data=go.Scatter3d(x=x[0,::nth], y=y[0,::nth], z=z[0,::nth],
                mode='markers',
                marker=dict(size=0.5)),
    layout=go.Layout(scene=dict(
                     xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
                     yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
                     zaxis=dict(range=[zm, zM], autorange=False, zeroline=False)),
                     scene_aspectmode='cube',
                     width=600, height=600,
                    updatemenus=[dict(type="buttons",
                                                buttons=[dict(label="Play",
                                                                method="animate",
                                                                args=[None, {"frame": {"duration": 80},}])])]),
    frames=[go.Frame(data=go.Scatter3d(x=x[k, ::nth], y=y[k, ::nth], z=z[k, ::nth],
                mode='markers',
                marker=dict(size=0.5))) for k in range(x.shape[0])]
)

# fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z,
#                   alphahull=5,
#                   opacity=0.4,
#                  color='cyan')])


In [ ]:
fig.show(validate=False)